In [ ]:
# Mount google drive where files are located
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Set up 
import os
from os import listdir
import keras

from keras import backend as K  
from keras import models
from keras import layers
from keras import optimizers

from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import one_hot
from tensorflow.keras.applications import xception
from tensorflow.keras.preprocessing import image

import gc
import glob

from PIL import Image
from imageio import imread

np.random.seed(5)

In [ ]:
# Directory where all images are located
listdir('/content/drive/MyDrive/Harper/3-C7082/Assignment/C7082-assignment/new-data')

['train', 'validation', 'test']

In [ ]:
# Define directories for training, validation and test images
base_path = '/content/drive/MyDrive/Harper/3-C7082/Assignment/C7082-assignment/new-data'

train_data_path = os.path.join(base_path, 'train/')
val_data_path = os.path.join(base_path, 'validation/')
test_data_path = os.path.join(base_path, 'test/')


In [ ]:
# Create object containing the list of species that make up the classes
species = listdir(train_data_path)
# And print to check they are all there!
print(species)

['bgs', 'chk', 'cls', 'cwd', 'cbl', 'wht', 'fhn', 'lsb', 'mze', 'smw', 'shp', 'sbt']


In [ ]:
# Creation of callbacks
# reduce_lr adjusts the learning rate
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss', # monitor validation loss for improvements
    factor = 0.5, # the factor that the learning rate will be multiplied by to decrease it
    patience = 3, # after 3 epochs the learning rate will be decreased
    min_lr = 0.0005, # the minimum learning rate that the model can go to
    verbose = 1) # set verbose to 1 so its visible

# early_stop stops training early if no improvements are seen
early_stop = tf.keras.callbacks.EarlyStopping(
              monitor = 'val_loss', # also monitor validation loss for improvements
              patience = 10, # patience of 10 epochs before early stopping
              verbose = 1) # make visible

In [ ]:
# Rescale images and apply data augmentations
train_datagen3 = ImageDataGenerator(
      rescale = 1/255, # this rescales the images so the values are between 0 and 1
      horizontal_flip = True, 
      vertical_flip = True,
      brightness_range = [0.5, 1.5]) # darkens or lightens the image each way by 50%

val_datagen3 = ImageDataGenerator(
    rescale = 1/255, 
      horizontal_flip = True, 
      vertical_flip = True,
      brightness_range = [0.5, 1.5])

# Do not apply data augmentation to test images
test_datagen = ImageDataGenerator(
    rescale = 1/255
)

In [ ]:
# Generate images for model
print("Total images in training data set:")
train_generator3 = train_datagen3.flow_from_directory(
    train_data_path,
    target_size = (299, 299),
    color_mode = "rgb",
    classes = species,
    class_mode = "categorical",
    batch_size = 64,
    shuffle = True,
    seed = 42
)

print("Total images in validation data set")
val_generator3 = val_datagen3.flow_from_directory(
    val_data_path,
    target_size = (299, 299),
    color_mode = "rgb",
    classes = species,
    class_mode = "categorical",
    batch_size = 64,
    shuffle = True,
    seed = 42
)


print("Total images in test data set")
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size = (299, 299),
    color_mode = "rgb",
    classes = species,
    class_mode = "categorical",
    batch_size = 1,
    shuffle = True,
    seed = 42
)

Total images in training data set:
Found 4939 images belonging to 12 classes.
Total images in validation data set
Found 300 images belonging to 12 classes.
Total images in test data set
Found 300 images belonging to 12 classes.


In [ ]:
# define pretrained Xception base
pretrained_base4 = xception.Xception(input_shape=[299, 299, 3],
                                     weights = 'imagenet',
                                     include_top=False)
pretrained_base4.trainable = False

83689472/83683744 [==============================] - 0s 0us/step


In [ ]:
pretrained_base4.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
# Create full model with new densely connected classifier
model6 = tf.keras.Sequential([
    pretrained_base4, # this is the pretrained Xception base from above 
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(256, activation = "relu"),
    tf.keras.layers.BatchNormalization(trainable = True, axis = -1),
    
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.BatchNormalization(trainable = True, axis = -1),

    tf.keras.layers.Dense(12, activation = 'softmax') # this should be number of classes?
])

In [ ]:
# Compile model with optimiser, loss and metrics
model6.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.005),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ['accuracy'])

In [ ]:
# Fit the model using the training data
result6 = model6.fit(train_generator3,
                    epochs = 50,
                    verbose = 1,
                    validation_data = val_generator3,
                    callbacks = [reduce_lr, early_stop])

Epoch 1/50
78/78 [==============================] - 5895s 76s/step - loss: 1.5982 - accuracy: 0.4847 - val_loss: 0.8979 - val_accuracy: 0.6833
Epoch 2/50
 4/78 [>.............................] - ETA: 29:17 - loss: 0.7477 - accuracy: 0.7402

KeyboardInterrupt: ignored

Increasing image size to 200x200x3 has increased training accuray to around 90% and validation accuracy to 84%.

In [ ]:
plt.plot(result6.history['accuracy'], label='train')
plt.plot(result6.history['val_accuracy'], label='valid')
plt.legend(loc='upper left')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

plt.plot(result6.history['loss'], label='train')
plt.plot(result6.history['val_loss'], label='test')
plt.legend(loc='upper right')
plt.title('Model Cost')
plt.ylabel('Cost')
plt.xlabel('Epoch')
plt.show()

In [ ]:
test_evaluate = model6.evaluate(test_generator)

In [ ]:
prediction = model6.predict_generator(test_generator,
                                     verbose = 1)

In [ ]:
predicted_class_indices = np.argmax(prediction, 
                                    axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]